Pytorch es una librería de Machine Learning inicialmente desarrollada por Facebook. Ofrece APIs para configurar Redes Neuronales Convolucionales, optimizar parametros y evaluar el rendimiento de un modelo.

In [ ]:
#Importamos los paquetes de PyTorch para crear CNN

import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

Por razones de simplicidad y pedagógicas, utilizaremos el dataset CIFAR10 ya existente en PyTorch: https://www.cs.toronto.edu/~kriz/cifar.html

CIFAR-10 consiste en 60.000 imágenes de 32x32 colores en 10 clases, con 6.000 imágenes por clase. Hay 50.000 imágenes de entrenamiento y 10.000 imágenes de prueba.

In [ ]:
#Preprocessing para cargar y preparar los datos
BatchSize=4

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

torchvision permite usar dataset clasicos para el tema de procesamiento de imagenes.

torchvision.datasets.$\textit{NombreDelDataset}$ permite usar algunos de los dataset más clasicos con respecto al procesamiento de imagenes. En este caso se usa CIFAR10

Con respecto a los parametros usados, hacen lo siguiente:

- root, indica en donde esta la data
- train, indica si el dataset que se usara es de training o el de testing
- download, si es que se descargara
- transform, la transformacion a ocupar, en caso de que se ocupe

Por otra parte DataLoader permite iterar sobre el dataset, y ver formas de muestreo:

- shuffle, permite randomizar el dataset

- num_workers, sirve para aumentar la rapidez con la cual se "alimenta" la red neuronal, mientras más, más ram se ocupa pero más rapido.

- batch_size, es con cuantos datos a la ves se alimentara a la red neuronal. Este parametro es super importante, dado que un tamaño optimo servira para que converga más rapido a una buena solucion. Usualmente , mientras más datos, más rapido, pero peor el resultado.




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# algunas funciones útiles para mostrar imagenes especificas del dataset

def plotImages(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

def imagesFromBatches(iterator,quantity):
    
    dataiter = iter(iterator)
    images, labels = dataiter.next()
    
    plotImages(torchvision.utils.make_grid(images))
    print(' '.join('%5s' % classes[labels[j]] for j in range(quantity)))
    return (images,labels)

images,labels=imagesFromBatches(trainloader,BatchSize)

# get some random training images
##dataiter = iter(trainloader)
##images, labels = dataiter.next()

# show images
##imshow(torchvision.utils.make_grid(images))
# print labels



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#Usando nn.Module, se puede comenzar a crear redes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)
    
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)
    

    def forward(self, x):
       
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # esta capa ayuda a reducir el espacio dimensional,a sola una capa 
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

summary(net,(3,32,32))




In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # se obtienen los parametros de la data de entrenamiento
        inputs, labels = data

        # se inicaliza en cero los gradientes para este mini-batch
        # Esto se hace para evitar la acumulacion del gradiente
        # Ya que el optimizador solo necesita el computo del mini-batch actual
        optimizer.zero_grad()

        outputs = net(inputs)
        
        # Se calcula la perdida, con el criterio seleccionado
        
        loss = criterion(outputs, labels)
        
        # Se obtienen los gradientes del parametro
        loss.backward()
        
        #Se acutalizan los gradientes obtenidos
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
images,labels= imagesFromBatches(testloader,BatchSize)

In [ ]:
outputs = net(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))